# Deploying Pipelines with Snowflake and dbt Labs

* Author: Tony Tawakali
* Last Updated: 5/08/2025

Welcome to the beginning of the Quickstart! Please refer to [the official Deploying Pipelines with Snowflake and dbt Labs
 Quickstart](https://quickstarts.snowflake.com/guide/data_engineering_with_notebooks/index.html?index=..%2F..index#0) for all the details including set up steps.

## Step 01 Setup Snowflake

During this step we will create our demo environment. Update the SQL variables below with your GitHub username and Personal Access Token (PAT) as well as with your forked GitHub repository information.


In [ ]:
SET MY_USER = CURRENT_USER();

SET GITHUB_SECRET_USERNAME = 'yourgithubusername';
SET GITHUB_SECRET_PASSWORD = 'yourgithubpassword';
SET GITHUB_URL_PREFIX = 'https://github.com/yourgithubusername';
SET GITHUB_REPO_ORIGIN = 'https://github.com/Snowflake-Labs/sfguide-deploying-pipielines-with-snowflake-and-dbt-labs.git';

In [ ]:
USE ROLE ACCOUNTADMIN;
CREATE DATABASE IF NOT EXISTS sandbox ;
USE DATABASE SANDBOX;

-- Secrets (schema level)
CREATE OR REPLACE SECRET DEMO_GITHUB_SECRET
  TYPE = password
  USERNAME = $GITHUB_SECRET_USERNAME
  PASSWORD = $GITHUB_SECRET_PASSWORD;

-- API Integration (account level)
-- This depends on the schema level secret!
CREATE OR REPLACE API INTEGRATION DEMO_GITHUB_API_INTEGRATION
  API_PROVIDER = GIT_HTTPS_API
  API_ALLOWED_PREFIXES = ($GITHUB_URL_PREFIX)
  ALLOWED_AUTHENTICATION_SECRETS = (DEMO_GITHUB_SECRET)
  ENABLED = TRUE;

-- Git Repository
CREATE OR REPLACE GIT REPOSITORY DEMO_GIT_REPO
  API_INTEGRATION = DEMO_GITHUB_API_INTEGRATION
  GIT_CREDENTIALS = DEMO_GITHUB_SECRET
  ORIGIN = $GITHUB_REPO_ORIGIN;

## Step 02 Setup dev environment


In [ ]:
USE ROLE ACCOUNTADMIN;
USE DATABASE SANDBOX;

ALTER GIT REPOSITORY DEMO_GIT_REPO FETCH;
EXECUTE IMMEDIATE FROM @DEMO_GIT_REPO/branches/main/scripts/deploy_environment.sql USING (env => 'DEV');

## Step 04 Setup prod environment


In [ ]:
USE ROLE ACCOUNTADMIN;
USE DATABASE SANDBOX;

ALTER GIT REPOSITORY DEMO_GIT_REPO FETCH;
EXECUTE IMMEDIATE FROM @DEMO_GIT_REPO/branches/main/scripts/deploy_environment.sql USING (env => 'PROD');

## Step 10 Teardown

Finally, we will tear down our demo environment.

In [ ]:
USE ROLE ACCOUNTADMIN;
DROP DATABASE dbt_hol_2025_dev;
DROP DATABASE dbt_hol_2025_prod;
